# 回帰分析による因果効果推定

## OLS回帰 (Ordinary Least Squares)

概要と基本用語

OLS回帰 は、連続アウトカムに対して、説明変数（治療や交絡因子）とアウトカムの間の線形関係を推定する手法。

- 回帰係数 ($\beta$)：各説明変数がアウトカムに与える影響の大きさ
- 切片 ($\beta_0$)：すべての説明変数が0のときのアウトカムの予測値
- 残差：実際の観測値 $Y_i$ と予測値 $\hat{Y_i}$ の差  $\epsilon_i = Y_i - \hat{Y_i}$

OLS 回帰モデルは以下の形で表される。

$Y_i = \beta_0 + \beta_1 X_{1i} + \beta_2 X_{2i} + \cdots + \beta_k X_{ki} + \epsilon_i$

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(42)
n = 10000

treatment = np.random.binomial(1, 0.5, n)
age = np.random.randint(20, 61, n)
base_outcome = 20 + age / 2 + np.random.normal(0, 5, n)
treatment_effect = treatment * 10
outcome = base_outcome + treatment_effect

df_ols = pd.DataFrame({
    'outcome': outcome,
    'treatment': treatment,
    'age': age
})

X = sm.add_constant(df_ols[['treatment', 'age']])
y = df_ols['outcome']

# この結果で、治療変数の係数 \beta_1 が概ね 10 付近であれば、治療効果が正しく捉えられていることが確認できる。
model_ols = sm.OLS(y, X).fit()
print(model_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                 1.231e+04
Date:                Mon, 17 Feb 2025   Prob (F-statistic):               0.00
Time:                        16:33:25   Log-Likelihood:                -30280.
No. Observations:               10000   AIC:                         6.057e+04
Df Residuals:                    9997   BIC:                         6.059e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.7749      0.183    108.029      0.0

## ロジスティック回帰

概要と基本用語

ロジスティック回帰 は、アウトカムが二値の場合に用いられ、説明変数がアウトカムの発生確率に与える影響を推定する。

- ロジット関数： $\text{logit}(p) = \ln\left(\frac{p}{1-p}\right)$ 
- オッズ比：治療の有無でアウトカム発生のオッズがどのように変化するかを示す指標

ロジスティック回帰モデルは以下の形で表される。

$\ln\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1 X_1 + \cdots + \beta_k X_k$

ここで p はアウトカムが1となる確率。

In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(42)
n = 10000

treatment = np.random.binomial(1, 0.5, n)
age = np.random.randint(20, 61, n)
logit = -5 + 0.1 * age + 1.5 * treatment
prob = 1 / (1 + np.exp(-logit))
outcome = np.random.binomial(1, prob)

df_logit = pd.DataFrame({
    'outcome': outcome,
    'treatment': treatment,
    'age': age
})

X_logit = sm.add_constant(df_logit[['treatment', 'age']])
y_logit = df_logit['outcome']

# このモデルでは、治療の係数オッズ比が、治療がアウトカムに与える効果を示す。
model_logit = sm.Logit(y_logit, X_logit).fit()
print(model_logit.summary())

Optimization terminated successfully.
         Current function value: 0.534822
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                outcome   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 17 Feb 2025   Pseudo R-squ.:                  0.2243
Time:                        17:49:51   Log-Likelihood:                -5348.2
converged:                       True   LL-Null:                       -6894.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.8393      0.107    -45.146      0.000      -5.049      -4.629
treatment      1.5157      0.

## 一般化線形モデル (GLM; Generalized Linear Model)

GLM は、OLS 回帰を拡張した手法で、アウトカムの分布が正規分布以外の場合でも柔軟に扱える統一的な枠組み。

- リンク関数 (Link Function)：説明変数の線形結合と、アウトカムの期待値 $\mu$ を結び付ける関数。たとえば、ロジスティック回帰ではロジットリンク（ $\text{logit}(p)$ ）、ポアソン回帰では対数リンク（ $\ln(\mu)$ ）が使われる。
- 分布族 (Exponential Family)：GLM ではアウトカムが従う確率分布として、正規分布、二項分布、ポアソン分布などを仮定。
- モデルの一般形
GLM は次のように表現される。

$g(\mu_i) = \beta_0 + \beta_1 X_{1i} + \cdots + \beta_k X_{ki}$

ここで、 $g(\cdot)$  がリンク関数、$\mu_i = E(Y_i)$ です。たとえば、ロジスティック回帰の場合、 $g(p) = \ln\left(\frac{p}{1-p}\right)$  となる。

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

np.random.seed(42)
n = 10000

treatment = np.random.binomial(1, 0.5, n)
age = np.random.randint(20, 61, n)
logit = -5 + 0.1 * age + 1.5 * treatment
prob = 1 / (1 + np.exp(-logit))
outcome = np.random.binomial(1, prob)

df_glm = pd.DataFrame({
    'outcome': outcome,
    'treatment': treatment,
    'age': age
})

# GLM を用いたロジスティック回帰
# family=sm.families.Binomial() で二項分布を指定
model_glm = smf.glm(formula="outcome ~ treatment + age", data=df_glm,
                    family=sm.families.Binomial()).fit()
print(model_glm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                outcome   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5348.2
Date:                Sun, 16 Feb 2025   Deviance:                       10696.
Time:                        16:49:26   Pearson chi2:                 9.99e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.2660
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.8393      0.107    -45.146      0.0

- リンク関数：例えばロジスティック回帰では、リンク関数 $g(p) = \ln\left(\frac{p}{1-p}\right)$ が使われ、モデルは $\ln\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1 \text{treatment} + \beta_2 \text{age}$ と表されます。
- 分布族：ここではアウトカムが二値なので、二項分布（Binomial）が適用されますが、GLM では他にもポアソン分布やガンマ分布などを指定でき、さまざまなデータの性質に合わせてモデルを構築できます。

# 出力の解釈

(OLS回帰の場合)

この結果は、OLS（普通最小二乗法）回帰モデルを用いて、説明変数として「treatment」と「age」を用い、目的変数「outcome」に対するモデルの適合度や各係数の推定結果を示している。回帰分析では、各説明変数が目的変数に与える影響の大きさ（係数）と、その係数の統計的有意性を検証するものである。

- Dep. Variable: outcome
被説明変数（目的変数）は outcome である。

- R-squared: 0.711
 決定係数（R²）は0.711であり、モデルが目的変数の変動の約71.1%を説明している。値が1に近いほど、モデルの説明力が高いことを示す。
	
- Adj. R-squared: 0.711
自由度調整済み決定係数であり、説明変数の数を考慮して調整された値で、ここではほぼ同じである。

- F-statistic: 1.231e+04
モデル全体が有意かどうかを評価する F 検定の統計量である。この値が大きいということは、少なくとも1つの説明変数が目的変数に有意な影響を与えていることを示す。

- Prob (F-statistic): 0.00
F統計量に対応する p 値は0.00であり、モデル全体が統計的に有意であると判断できる。

- AIC: 60570 および BIC: 60590
これらはモデル選択の指標であり、値が小さいほど良いモデルとされる。ただし、これらは他のモデルと比較する際に用いる。

### 各説明変数の推定結果

以下の表は、各説明変数の推定結果を示している。

| 変数       | coef    | std err  | t       | P>t    | [0.025, 0.975]           |
|---|---|---|---|---|-----|
| const      | 19.7749 | 0.183    | 108.029 | 0.000   | 19.416 ～ 20.134          |
| treatment  | 10.2499 | 0.100    | 102.514 | 0.000   | 10.054 ～ 10.446          |
| age        | 0.5041  | 0.004    | 118.935 | 0.000   | 0.496 ～ 0.512            |

const (切片)
- 切片の係数は 19.7749 であり、全ての説明変数が 0 の場合に予測される outcome の値は約19.77である。
- 標準誤差は 0.183 であり、これは推定の不確かさを表す指標である。
- t値が 108.029、p値が 0.000 であることから、切片は統計的に有意であると判断される。

treatment
- treatment の係数は 10.2499 であり、治療群の場合、平均でアウトカムが約10.25増加することを示している。
- 標準誤差は 0.100 で、t値は 102.514、p値は 0.000 であるため、この効果は非常に有意である。
- 95% 信頼区間は 10.054 から 10.446 であり、真の効果がこの範囲にあると95%の信頼性で言える。

age
- age の係数は 0.5041 であり、年齢が1歳増加するごとにアウトカムは約0.50増加することを示している。
- 標準誤差は 0.004 と非常に小さく、t値は 118.935、p値は 0.000 であるため、統計的に非常に有意な影響がある。
- 信頼区間は 0.496 から 0.512 であり、推定値は非常に精度が高い。

残差と診断統計量
- Omnibus: 0.085, Prob(Omnibus): 0.958
残差の正規性を検定する指標であり、p値が0.958と非常に高いため、残差は正規分布に従っているとみなせる。
- Durbin-Watson: 2.023
自己相関の検定指標であり、値が約2に近いため、残差間に自己相関はほぼないと判断される。
- Jarque-Bera (JB): 0.106, Prob(JB): 0.948
こちらも残差の正規性を検定する統計量であり、p値が0.948であるため、残差は正規分布に従っていると評価できる。
- Skew: 0.000, Kurtosis: 2.984
残差の歪度と尖度を示しており、理想的な正規分布では歪度は0、尖度は3に近い。今回の値は正規性の仮定とほぼ整合している。
- Cond. No.: 155
説明変数間の共線性の指標であり、一般に 10～30 以上で注意が必要だが、ここでは 155 と示されているが、大きな問題ではないと考えられる（ただしケースバイケースで判断する）。